Generate training and testing dataset

In [ ]:
from Preprocessing.Preprocessor import preprocess,split_dataset


preprocess("./inputdata.czi","./preprocessed",256,256,20,148,True,99.8,3,2)

split_dataset("./preprocessed",100)

Train VM-3D model

In [ ]:
from train import train, create_path_names
from torch.utils.tensorboard import SummaryWriter

output = "%s/%s" % ("./", "modelname")
dataset_name = "preprocessed"


create_path_names(output, dataset_name)
tensorboard_writer = SummaryWriter(log_dir="%s%s/tb" % (output, dataset_name))

train(epoch=0, n_epochs=200, pathname="./", output_path=output, dataset_name=dataset_name,
      batch_size=2, glr=0.0002,dlr=0.0002, b1=0.5, b2=0.999, n_cpu=8, img_depth=128, img_width=256, img_height=256, channels=2, sample_interval=1,checkpoint_interval=10, lambda_voxel=100, tensorboard_writer=tensorboard_writer)

Load and Predict using virtual multiplexing model

In [ ]:
from predict import load_model, predict
from Model.dataset import DataLoader, PredictDataset
import os

generator = load_model("./modelname", channels = 2)

if os.path.exists("./output_path") == False:
    os.mkdir("./output_path")

dl = DataLoader(
        PredictDataset("./preprocessing/validation"),
        batch_size=1,
        shuffle=False,
        num_workers=1,
    )

predict(dl, generator, "./output_path")
